# Starter Notebook

Install and import required libraries

In [1]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
     ---------------------------------------- 0.8/75.4 MB 3.7 MB/s eta 0:00:20
      --------------------------------------- 1.3/75.4 MB 3.4 MB/s eta 0:00:23
     - -------------------------------------- 2.1/75.4 MB 3.3 MB/s eta 0:00:23
     - -------------------------------------- 2.6/75.4 MB 3.4 MB/s eta 0:00:22
     - -------------------------------------- 3.4/75.4 MB 3.1 MB/s eta 0:00:23
     -- ------------------------------------- 4.2/75.4 MB 3.4 MB/s eta 0:00:21
     -- ------------------------------------- 5.0/75.4 MB 3.5 MB/s eta 0:00:21
     --- ------------------------------------ 6.0/75.4 MB 3.5 MB/s eta 0:00:20
     --- ------------------------------------ 6.8/75.4 MB 3.6 MB/s eta 0:00:20
     ---- ----------------------------------- 7.6/75.4 MB 3.6 MB/s eta 0:00:19
     ---- ----------------------------------- 8.7/75.4 MB 3.8 MB/s eta 0:00:1

In [1]:
import os
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel
import pickle


d:\conda\envs\accelerate\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Tokenizer and Preprocess Data

In [2]:
base_model = 'roberta-base'

dataset = load_dataset('ag_news', split='train')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

def preprocess(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized

tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [28]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(eval_dataset)}")
print(f"Unlabelled dataset size: {len(unlabelled_dataset)}")

Train dataset size: 119360
Test dataset size: 640
Unlabelled dataset size: 8000


In [3]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


#Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [4]:
model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

#Anything from here on can be modified

In [5]:
# Split the original training set
split_datasets = tokenized_dataset.train_test_split(test_size=640, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

#Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [14]:
# Setup PEFT config and get PEFT model for fine-tuning

# PEFT Config
peft_config = LoraConfig(
    r=7,  # Reduce rank to decrease parameters
    lora_alpha=16,  # Keep a high alpha value
    lora_dropout=0.1,  # Keep dropout unchanged
    bias='none',
    # Reduce target modules
    target_modules=['query', 'key', 'value'],
    task_type="SEQ_CLS",
    # Add module saving for training the classifier head
    modules_to_save=["classifier"],
)


In [15]:
peft_model = get_peft_model(model, peft_config)
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [16]:
print("Trainable parameters:")
for name, param in peft_model.named_parameters():
    if param.requires_grad:
        print(name)

Trainable parameters:
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.key.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.key.lora_B.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.lora_B.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.key.lora_A.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.key.lora_B.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.value.lora_A.default.weight
base_model.model.roberta.encoder.layer.1.attention.self.value.

In [17]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 980,740 || all params: 125,629,448 || trainable%: 0.7807


## Training Setup

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [19]:
from transformers import get_scheduler

# Setup Training args
output_dir = "results"
training_args = TrainingArguments(
    output_dir=output_dir,
    report_to="none",
    eval_strategy='steps',
    eval_steps=100,
    logging_steps=50,
    save_strategy="steps",
    save_steps=100,
    learning_rate=5e-5,
    num_train_epochs=5,
    max_steps=-1,
    warmup_ratio=0.1,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    use_cpu=False,
    dataloader_num_workers=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    optim="adamw_torch",
    gradient_accumulation_steps=2,
    fp16=True,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True},

    metric_for_best_model="accuracy",
    load_best_model_at_end=True
)

def get_trainer(model):
    return Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

### Start Training

In [20]:
peft_lora_finetuning_trainer = get_trainer(peft_model)

# Add early stopping callback to prevent overfitting
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=5)
peft_lora_finetuning_trainer.add_callback(early_stopping)

result = peft_lora_finetuning_trainer.train()

# Save the best model
peft_lora_finetuning_trainer.save_model("best_model")

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,1.391000,1.387672,0.246875,0.097760,0.060947,0.246875
200,1.378100,1.370430,0.450000,0.352958,0.613511,0.450000
300,1.310700,1.255884,0.737500,0.723503,0.795555,0.737500
400,0.550300,0.363881,0.881250,0.881014,0.882825,0.881250
500,0.342700,0.322025,0.895312,0.895255,0.896047,0.895312
600,0.313300,0.308560,0.906250,0.906092,0.906853,0.906250
700,0.279400,0.292819,0.906250,0.905848,0.906768,0.906250
800,0.289500,0.305858,0.904687,0.904184,0.906381,0.904687
900,0.288300,0.292079,0.906250,0.905999,0.907023,0.906250
1000,0.287200,0.287599,0.904687,0.904412,0.905226,0.904687


## Evaluate Finetuned Model


In [21]:
# Compute and print the final evaluation metrics
final_eval = peft_lora_finetuning_trainer.evaluate()
print(f"Final evaluation metrics: {final_eval}")

# Check the number of model parameters to ensure it is within 1 million
total_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params}")

Final evaluation metrics: {'eval_loss': 0.3085595369338989, 'eval_accuracy': 0.90625, 'eval_f1': 0.9060916288581531, 'eval_precision': 0.9068527980239759, 'eval_recall': 0.90625, 'eval_runtime': 25.9713, 'eval_samples_per_second': 24.643, 'eval_steps_per_second': 0.385, 'epoch': 0.5898123324396782}
Total trainable parameters: 980740


### Performing Inference on Custom Input
Uncomment following functions for running inference on custom inputs

In [22]:
def classify(model, tokenizer, text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
    output = model(**inputs)
    prediction = output.logits.argmax(dim=-1).item()
    print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
    return id2label[prediction]

In [23]:
classify( peft_model, tokenizer, "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( peft_model, tokenizer, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")


 Class: 1, Label: Sports, Text: Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...

 Class: 2, Label: Business, Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlinand of ultra-cynics, are seeing green again.


'Business'

### Run Inference on eval_dataset

In [24]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=8, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

In [25]:
# Check evaluation accuracy
_, _ = evaluate_model(peft_model, eval_dataset, True, 8, data_collator)

100%|██████████| 80/80 [00:02<00:00, 35.97it/s]

Evaluation Metric: {'accuracy': 0.90625}


### Run Inference on unlabelled dataset

In [26]:
#Load your unlabelled data
unlabelled_dataset = pd.read_pickle("test_unlabelled.pkl")
test_dataset = unlabelled_dataset.map(preprocess, batched=True, remove_columns=["text"])
unlabelled_dataset

Map: 100%|██████████| 8000/8000 [00:04<00:00, 1816.97 examples/s]


Dataset({
    features: ['text'],
    num_rows: 8000
})

In [27]:
# Run inference and save predictions
preds = evaluate_model(peft_model, test_dataset, False, 8, data_collator)
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()  # or preds.tolist()
})
df_output.to_csv(os.path.join(output_dir,"inference_output.csv"), index=False)
print("Inference complete. Predictions saved to inference_output.csv")

100%|██████████| 1000/1000 [00:19<00:00, 50.67it/s]

Inference complete. Predictions saved to inference_output.csv
